# Connecting our notebook with ClearMl Server


In [1]:
%env CLEARML_WEB_HOST=https://app.clear.ml
%env CLEARML_API_HOST=https://api.clear.ml
%env CLEARML_FILES_HOST=https://files.clear.ml
%env CLEARML_API_ACCESS_KEY=YWY1GHK2BDMDJAAZF6N4
%env CLEARML_API_SECRET_KEY=WFUUuS2Ziq8LRMCh2TdWJTbw2iuVz6ZyQa2PBcEWWYD0odAIHH

env: CLEARML_WEB_HOST=https://app.clear.ml
env: CLEARML_API_HOST=https://api.clear.ml
env: CLEARML_FILES_HOST=https://files.clear.ml
env: CLEARML_API_ACCESS_KEY=YWY1GHK2BDMDJAAZF6N4
env: CLEARML_API_SECRET_KEY=WFUUuS2Ziq8LRMCh2TdWJTbw2iuVz6ZyQa2PBcEWWYD0odAIHH


# Import library


In [17]:
import torch
from torch_geometric.datasets import ShapeNet
from torch_geometric.loader import DataLoader


import plotly.express as px

## Datasets

In [3]:
data = ShapeNet(root='./datasets',categories=['Bag','Cap',"Car","Chair"]).shuffle()[:500]

In [4]:
print("Number of Samples",len(data))
print("Sample",data[0])

Number of Samples 500
Sample Data(x=[2541, 3], y=[2541], pos=[2541, 3], category=[1])


In [5]:
data[1].pos

tensor([[ 0.4262,  0.0109, -0.1247],
        [-0.3352, -0.0009, -0.1304],
        [ 0.3119,  0.0482, -0.0839],
        ...,
        [ 0.3016, -0.0158,  0.1137],
        [-0.3376, -0.0534,  0.1167],
        [ 0.2607,  0.0968,  0.0132]])

In [6]:
cat_dict = {key: 0 for key in data.categories}
for datapoint in data:cat_dict[data.categories[datapoint.category.int()]]+=1
cat_dict

{'Bag': 7, 'Cap': 3, 'Car': 95, 'Chair': 395}

Data(x=[2535, 3], y=[2535], pos=[2535, 3], category=[1])

In [23]:

def plot_3d(shape):
    '''This function create point 3d model from shape '''

    print('Number of data points',shape.x.shape[0])
    print('Type:',shape.category)
    
    x = shape.pos[:,0]
    y = shape.pos[:,1]
    z = shape.pos[:,2]

    fig = px.scatter_3d(x=x,y=y,z=z)
    fig.show()

plot_3d(data[1])


Number of data points 2535
Type: tensor([2])


# Model

In [8]:
#https://www.kaggle.com/code/balraj98/pointnet-for-3d-object-classification-pytorch
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F

class Tnet(nn.Module):
    def __init__(self, k=3):
        super().__init__()
        self.k=k
        self.conv1 = nn.Conv1d(k,64,1)
        self.conv2 = nn.Conv1d(64,128,1)
        self.conv3 = nn.Conv1d(128,1024,1)
        self.fc1 = nn.Linear(1024,512)
        self.fc2 = nn.Linear(512,256)
        self.fc3 = nn.Linear(256,k*k)

        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)
        self.bn4 = nn.BatchNorm1d(512)
        self.bn5 = nn.BatchNorm1d(256)


    def forward(self, input):
        # input.shape == (bs,n,3)
        bs = input.size(0)
        xb = F.relu(self.bn1(self.conv1(input)))
        xb = F.relu(self.bn2(self.conv2(xb)))
        xb = F.relu(self.bn3(self.conv3(xb)))
        pool = nn.MaxPool1d(xb.size(-1))(xb)
        flat = nn.Flatten(1)(pool)
        xb = F.relu(self.bn4(self.fc1(flat)))
        xb = F.relu(self.bn5(self.fc2(xb)))

        #initialize as identity
        init = torch.eye(self.k, requires_grad=True).repeat(bs,1,1)
        if xb.is_cuda:
            init=init.cuda()
        matrix = self.fc3(xb).view(-1,self.k,self.k) + init
        return matrix


class Transform(nn.Module):
    def __init__(self):
        super().__init__()
        self.input_transform = Tnet(k=3)
        self.feature_transform = Tnet(k=64)
        self.conv1 = nn.Conv1d(3,64,1)

        self.conv2 = nn.Conv1d(64,128,1)
        self.conv3 = nn.Conv1d(128,1024,1)


        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)

    def forward(self, input):
        matrix3x3 = self.input_transform(input)
        # batch matrix multiplication
        xb = torch.bmm(torch.transpose(input,1,2), matrix3x3).transpose(1,2)

        xb = F.relu(self.bn1(self.conv1(xb)))

        matrix64x64 = self.feature_transform(xb)
        xb = torch.bmm(torch.transpose(xb,1,2), matrix64x64).transpose(1,2)

        xb = F.relu(self.bn2(self.conv2(xb)))
        xb = self.bn3(self.conv3(xb))
        xb = nn.MaxPool1d(xb.size(-1))(xb)
        output = nn.Flatten(1)(xb)
        return output, matrix3x3, matrix64x64

class PointNet(nn.Module):
    def __init__(self, classes = 10):
        super().__init__()
        self.transform = Transform()
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, classes)
        

        self.bn1 = nn.BatchNorm1d(512)
        self.bn2 = nn.BatchNorm1d(256)
        self.dropout = nn.Dropout(p=0.3)
        self.logsoftmax = nn.LogSoftmax(dim=1)

    def forward(self, input):
        xb, matrix3x3, matrix64x64 = self.transform(input)
        xb = F.relu(self.bn1(self.fc1(xb)))
        xb = F.relu(self.bn2(self.dropout(self.fc2(xb))))
        output = self.fc3(xb)
        return self.logsoftmax(output), matrix3x3, matrix64x64
        

In [9]:
def pointnetloss(outputs, labels, m3x3, m64x64, alpha = 0.0001):
    criterion = torch.nn.NLLLoss()
    bs=outputs.size(0)
    id3x3 = torch.eye(3, requires_grad=True).repeat(bs,1,1)
    id64x64 = torch.eye(64, requires_grad=True).repeat(bs,1,1)
    if outputs.is_cuda:
        id3x3=id3x3.cuda()
        id64x64=id64x64.cuda()
    diff3x3 = id3x3-torch.bmm(m3x3,m3x3.transpose(1,2))
    diff64x64 = id64x64-torch.bmm(m64x64,m64x64.transpose(1,2))
    return criterion(outputs, labels) + alpha * (torch.norm(diff3x3)+torch.norm(diff64x64)) / float(bs)

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [11]:
pointnet = PointNet()
pointnet.to(device);

In [12]:
optimizer = torch.optim.Adam(pointnet.parameters(), lr=0.00025)

In [13]:
def train(model, train_loader, val_loader=None,  epochs=4):
    for epoch in range(epochs): 
        pointnet.train()
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data['pointcloud'].to(device).float(), data['category'].to(device)
            optimizer.zero_grad()
            outputs, m3x3, m64x64 = pointnet(inputs.transpose(1,2))

            loss = pointnetloss(outputs, labels, m3x3, m64x64)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % 5 == 4:    # print every 10 mini-batches
                print('[Epoch: %d, Batch: %4d / %4d], loss: %.3f' %
                    (epoch + 1, i + 1, len(train_loader), running_loss / 10))
                running_loss = 0.0

        pointnet.eval()
        correct = total = 0

        # validation
        if val_loader:
            with torch.no_grad():
                for data in val_loader:
                    inputs, labels = data['pointcloud'].to(device).float(), data['category'].to(device)
                    outputs, __, __ = pointnet(inputs.transpose(1,2))
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
            val_acc = 100. * correct / total
            print('Valid accuracy: %d %%' % val_acc)

        # save the model
        torch.save(pointnet.state_dict(), "save.pth")

In [39]:
data[1]


Data(x=[2535, 3], y=[2535], pos=[2535, 3], category=[1])

In [15]:
data.pos

tensor([[ 0.1939, -0.3420, -0.2628],
        [-0.0475,  0.0016, -0.2269],
        [-0.1342,  0.2142, -0.1732],
        ...,
        [-0.1271, -0.0184,  0.1391],
        [-0.1098,  0.1016,  0.1223],
        [-0.1445,  0.4083,  0.1621]])

In [64]:
data_loader = DataLoader(data,batch_size=1,shuffle=True)

In [65]:
data_loader

In [66]:
def train(model, train_loader, val_loader=None,  epochs=4):
    for epoch in range(epochs): 
        pointnet.train()
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data.pos.to(device).float(), data.category.to(device)
            optimizer.zero_grad()
            outputs, m3x3, m64x64 = pointnet(inputs)


            loss = pointnetloss(outputs, labels, m3x3, m64x64)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % 5 == 4:    # print every 10 mini-batches
                print('[Epoch: %d, Batch: %4d / %4d], loss: %.3f' %
                    (epoch + 1, i + 1, len(train_loader), running_loss / 10))
                running_loss = 0.0

        pointnet.eval()
        correct = total = 0

        # validation
        if val_loader:
            with torch.no_grad():
                for data in val_loader:
                    inputs, labels = data['pointcloud'].to(device).float(), data['category'].to(device)
                    outputs, __, __ = pointnet(inputs.transpose(1,2))
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
            val_acc = 100. * correct / total
            print('Valid accuracy: %d %%' % val_acc)

        # save the model
        torch.save(pointnet.state_dict(), "save.pth")

In [67]:
train(pointnet, data_loader)

RuntimeError: Given groups=1, weight of size [64, 3, 1], expected input[1, 2790, 3] to have 3 channels, but got 2790 channels instead